# Solving ordinary differential equations

Ordinary differential equations (ODEs) are widely used in science and engineering, in particular for modeling dynamic
processes. While simple ODEs can be solved with analytical methods, non-linear ODEs are generally not possible
to solve in this way, and we need to apply numerical methods. In this chapter we will see how we can program
general numerical solvers that can be applied to any ODE. We will first consider scalar ODEs, i.e., ODEs with a single
equation and a single unknown, and in Chapter 3 we will
extend the ideas to systems of coupled ODEs. Understanding the concepts
of this chapter is useful not only for programming your own ODE solvers, but also for using a wide variety of
general-purpose ODE solvers available both in Python and other programming languages.


# Creating a general-purpose ODE solver
When solving ODEs analytically one will typically consider a specific ODE or a class of ODEs, and try to derive
a formula for the solution. In this chapter we want to implement numerical solvers that can be applied to any ODE,
not restricted to a single example or a particular class of equations. For this purpose, we need a general abstract
notation for an arbitrary ODE. We will write the ODEs on the following form:

<!-- Equation labels as ordinary links -->
<div id="ode0"></div>

$$
\begin{equation}
u^{\prime}(t) = f(u(t), t),
\label{ode0} \tag{1}
\end{equation}
$$

which means that the ODE is fully specified by the definition of the right hand side function $f(u,t)$. Examples of this
function may be:

$$
\begin{align*}
f(u,t) &= \alpha u,\quad\hbox{exponential growth}\\
f(u,t) &= \alpha u\left(  1-\frac{u}{R}\right),\quad\hbox{logistic growth}\\
f(u,t) &= -b|u|u + g,\quad\hbox{falling body in a fluid}
\end{align*}
$$

Notice that for generality we write all these right hand sides as functions of both $u$ and $t$, although the
mathematical formulations only involve $u$. It will become clear later why such a general formulation is useful.
Our aim is now to write functions and classes that take $f$ as input, and solve the corresponding ODE to produce
$u$ as output.

### The Euler method turns an ODE into a difference equation.

All the numerical methods we will considered in this chapter are based on approximating the derivatives
in the equation $u'=f(u,t)$ by finite differences. This step transforms the ODE into
a difference equation, which can be solved with the tools presented in Chapter 1.
To introduce the idea, assume that we have computed $u$ at discrete time points
$t_0,t_1,\ldots,t_n$. At time $t_n$ we have the ODE

$$
u'(t_n) = f(u(t_n),t_n),
$$

and we can now approximate $u'(t_n)$ by a forward finite difference;

$$
u'(t_n)\approx \frac{u(t_{n+1})-u(t_n)}{\Delta t} .
$$

Inserting this approximation into the ODE at $t=t_n$ yields the following equation

$$
\frac{u(t_{n+1})-u(t_n)}{\Delta t} = f(u(t_n),t_n),
$$

which we may recognize as a difference equation for computing $u(t_{n+1})$ from the known value
$u(t_n)$. We can rearrange the terms to obtain an explicit formula for $u(t_{n+1})$:

$$
u(t_{n+1}) = u(t_n) + \Delta t f(u(t_n), t_n) .
$$

This is known as the *Forward Euler (FE) method*, and is the simplest numerical method for solving
and ODE. We can simplify the
formula by using the notation for difference equations introduced in Chapter 1.
If we let $u_n$ denote the numerical approximation to the exact solution $u(t)$
at $t=t_n$, the difference equation can be written as

<!-- Equation labels as ordinary links -->
<div id="forw_euler"></div>

$$
\begin{equation} u_{n+1} = u_n + \Delta t f(u_n,t_n) .
\label{forw_euler} \tag{2}
\end{equation}
$$

This is a regular difference equation which can be solved using arrays and a for-loop, just as we did for the
other difference equations in Chapter 1. We
start from the known initial condition  $u_0$, and apply the formula repeatedly
to compute $u_1$, $u_2$, $u_3$ and so forth.

### An ODE needs an initial condition.

In mathematics, an initial condition for $u$ must be specified to have a unique solution of equation
([1](#ode0)). When solving the equation numerically, we need to set $u_0$ in order to start our method and
compute a solution at all. As an example, consider the very simple ODE

$$
u'=u .
$$

This equation has the general solution $u=Ce^t$ for any constant $C$, so it has an infinite number of solutions.
Specifying an initial condition $u(0)=u_0$ gives $C=u_0$, and we get the unique solution $u=u_0e^t$. When solving the
equation numerically, we start from our known $u_0$, and apply formula ([2](#forw_euler)) repeatedly:

$$
\begin{align*}
u_1 &= u_0 + \Delta t u_0, \\
u_2 &= u_1 + \Delta t u_1, \\
u_3 &= u_2 + \ldots .
\end{align*}
$$

Just as for the difference equations solved in the previous chapter, this repeated application
of the same formula is conveniently implemented in a for-loop. For a given time
step $\Delta t$ (`dt`) and number of time steps $n$, we perform the following steps:
1. Create arrays `t` and `u` of length $n+1$

2. Set initial condition: `u[0]` = $U_0$, `t[0]=0`

3. For $k=0,1,2,\ldots,n-1$:

  * `t[n+1] = t[n] + dt`

  * `u[n+1] = (1 + dt)*u[n]`


A complete Python implementation of this algorithm may look like

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

dt = 0.2
U0 = 1
T = 4
N = int(T/dt)

t = np.zeros(N+1)
u = np.zeros(N+1)

t[0] = 0
u[0] = U0
for n in range(N):
    t[n+1] = t[n] + dt
    u[n+1] = (1 + dt)*u[n]

plt.plot(t,u)
plt.show()

The solution is shown in [Figure](#fig:ode0), for two different choices of the time step $\Delta t$. We see that the
approximate solution improves as $\Delta t$ is reduced, although both the solutions are quite inaccurate. However, reducing the
time step further will easily create a solution that cannot be distinguished from the exact solution.

<!-- dom:FIGURE: [./fig-ode2/FE_n_10_20.png, width=600 frac=1] Solution of $u' = u, u(0) = 1$ with $\Delta t = 0.4$ and $\Delta t=0.2$. <div id="fig:ode0"></div> -->
<!-- begin figure -->
<div id="fig:ode0"></div>

<p>Solution of $u' = u, u(0) = 1$ with $\Delta t = 0.4$ and $\Delta t=0.2$.</p>
<img src="./fig-ode2/FE_n_10_20.png" width=600>

<!-- end figure -->



### Extending the solver to a general ODE.

As stated above, the purpose of this chapter is to create general-purpose ODE solvers, that can solve any ODE written
on the form $u'=f(u,t)$. This requires a very small modification of the algorithm above;
1. Create arrays `t` and `u` of length $n+1$

2. Set initial condition: `u[0]` = $u_0$, `t[0]=0`

3. For $k=0,1,2,\ldots,n-1$:

  * `u[n+1] = u[n] + dt*f(u[n], t[n])`

  * `t[n+1] = t[n] + dt`


The only change of the algorithm is in the formula for computing `u[n+1]` from `u[n]`.
In the previous case we had $f(u,t) = u$, and to create a
general-purpose ODE solver we simply replace `u[n]` with the more general `f(u[n],t[n])`.
The following Python function implements this generic version of the FE method:

In [2]:
def ForwardEuler(f, U0, T, N):
    """Solve u'=f(u,t), u(0)=U0, with n steps until t=T."""
    import numpy as np
    t = np.zeros(N+1)
    u = np.zeros(N+1)  # u[n] is the solution at time t[n]

    u[0] = U0
    t[0] = 0
    dt = T/N

    for n in range(N):
        t[n+1] = t[n] + dt
        u[n+1] = u[n] + dt*f(u[n], t[n])

    return u, t

This simple function can solve any ODE written on the form ([1](#ode0)). The right
hand side function $f(u,t)$ needs to be implemented as a Python function, and
then passed as an argument to `ForwardEuler` together with the initial condition, the
stop time `T` and the number of time steps. The two latter arguments are then used
to calculate the time step `dt` inside the function.

To illustrate how the function is used, let us apply it to solve the same problem as above;
$u'=u$, $u(0)=1$, for $t\in [0,4]$. The following code uses the `ForwardEuler` function to solve this problem:

In [3]:
def f(u, t):
    return u

U0 = 1
T = 3
N = 30
u, t = ForwardEuler(f, U0, T, N)
plt.plt(t,u)
plt.show()

The `ForwardEuler` function returns the two arrays `u` and `t`, which we can
plot or process further as we want. One thing worth noticing in this code is the definition of the
right hand side function `f`. As we mentioned above, this function should always be written
with two arguments `u` and `t`, although in this case only `u` is used inside the function.
The two arguments are needed because we want our solver to work for all ODEs on the
form $u' = f(u,t)$, and the function is therefore called as `f(u[n], t[n])` inside
`ForwardEuler`. If our right
hand side function was defined as `def f(u):` we would get an error message when the
function was called inside `ForwardEuler`. This problem is solved by simply writing
`def f(u,t):` even if `t` is never used inside the function.

: This way of defining the right hand side is a standard used by most
available ODE solver libraries, both in Python and other languages. The right hand
side function always takes two arguments `u` and `t`, but, annoyingly, the order of the
two arguments varies between different solver libraries. Some expect the `t` argument first,
while others expect `u` first.

For being only 15 lines of Python code, the capabilities of the `ForwardEuler`
function above are quite remarkable. Using this function, we can solve any
kind of linear or non-linear ODE, most of which would be impossible to solve
using analytical techniques. The general recipe goes as follows:
1. Identify $f(u,t)$ in your ODE

2. Make sure you have an initial condition $u_0$

3. Implement the $f(u,t)$ formula in a Python function `f(u, t)`

4. Choose the number of time steps `N`

5. Call `u, t = ForwardEuler(f, U0, T, N)`

6. Plot the solution

It is worth mentioning that the FE method is the simplest of all
ODE solvers, and many will argue that it is not very good. This is partly true,
since there are many other methods that are more accurate and more stable when applied to
challenging ODEs. We shall look at a few examples of such methods later in this
chapter. However, the FE method is quite suitable for solving most ODEs.
If we are not happy with the accuracy we can simply reduce the time step, and
in most cases this will give the accuracy we need with a negligible increase in computing time.



# The ODE solver implemented as a class
We can increase the flexibility of the `ForwardEuler`
solver function by implementing it as a class. The usage of the class
may for instance be as follows:

```Python
        method = ForwardEuler_v1(f, U0=0.0, T=40, N=400)
        u, t = method.solve()
        plot(t, u)
```

The benefits of using a class instead of a function may not be obvious at this
point, but it will become clear later. For now,
let us just look at how such a solver class can be implemented:
  * We need a constructor (`__init__`) which takes $f$, `T`, `N`, and `U0` as arguments

  and stores them as attributes.
  * The time step $\Delta t$ and the sequences $u_n$, $t_n$ must be

  initalized and stored as attributes. These tasks are also natural to handle in the constructor.
  * The class needs a `solve`-method, which implements the for-loop and returns

  the solution, similar to the `ForwardEuler` function considered above.

In addition to these methods, it may be convenient to implement the formula for
advancing the solution one step as a separate method `advance`. In this way it
becomes very easy to implement new numerical methods, since we typically only
need to change the `advance` method. A first version of the solver class may
look as follows:

In [4]:
import numpy as np

class ForwardEuler_v1:
    def __init__(self, f, U0, T, N):
        self.f, self.U0, self.T, self.N = f, U0, T, N
        self.dt = T/float(N)
        self.u = np.zeros(self.N+1)
        self.t = np.zeros(self.N+1)

    def solve(self):
        """Compute solution for 0 <= t <= T."""
        self.u[0] = float(self.U0)
        self.t[0] = float(0)

        for n in range(self.N):
            self.n = n
            self.t[n+1] = self.t[n] + self.dt
            self.u[n+1] = self.advance()
        return self.u, self.t

    def advance(self):
        """Advance the solution one time step."""
        # Create local variables to get rid of "self." in
        # the numerical formula
        u, dt, f, n, t = self.u, self.dt, self.f, self.n, self.t

        unew = u[n] + dt*f(u[n], t[n])
        return unew

This class does essentially the same tasks as the `ForwardEuler` function above.
The main advantage of the class implementation is
the increased flexibility that comes with the `advance` method. As we shall see
later, implementing a different numerical method typically only requires
implementing a new version of this method, while all the other code can be left unchanged.

We can also use a class to hold the right-hand side $f(u,t)$, which is
particularly convenient for functions with parameters.
Consider for instance the model for logistic growth;

$$
u^{\prime}(t)=\alpha u(t)\left(  1-\frac{u(t)}{R}\right),\quad u(0)=U_0,\quad t\in [0,40],
$$

which is the ODE version of the difference equation considered in Chapter 1.
The right hand side function has two parameters $\alpha$ and $R$, but if we
want to solve it using our `ForwardEuler` function or class, it must be implemented
as a function of $u$ and $t$ only. As we have discussed earlier in the course, %IN1900
a class with a call method provides a very flexible implementation of such a
function, since we can set the parameters as attributes in the constructor and
use them inside the `__call__` method:

In [5]:
class Logistic:
    def __init__(self, alpha, R, U0):
        self.alpha, self.R, self.U0 = alpha, float(R), U0

    def __call__(self, u, t):   # f(u,t)
        return self.alpha*u*(1 - u/self.R)

The main program for solving the logistic growth problem may now look like:

In [6]:
problem = Logistic(0.2, 1, 0.1)
method = ForwardEuler_v1(problem,problem.U0,40,401)
u, t = method.solve()
plt.plot(t,u)
plt.show()

<!-- dom:FIGURE: [./fig-ode2/logistic_func_mpl.png, width=600 frac=1.0] Solution of the logistic growth model. -->
<!-- begin figure -->

<p>Solution of the logistic growth model.</p>
<img src="./fig-ode2/logistic_func_mpl.png" width=600>

<!-- end figure -->


### An alternative class implementation of the FE method.

As always in programming, there are multiple ways to perform the same task, and
the `ForwardEuler_v1` class presented above is by no means the only possible class implementation of the
FE method. A possible alternative implementation is

In [7]:
import numpy as np

class ForwardEuler_v2:
    def __init__(self, f):
        self.f = f

    def set_initial_condition(self,U0):
        self.U0 = float(U0)

    def solve(self, time_points):
        """Compute solution for array of time points"""
        self.t = np.asarray(time_points)
        N = len(self.t)
        self.u = np.zeros(N)
        self.u[0] = self.U0

        # Time loop
        for n in range(N-1):
            self.n = n
            self.u[n+1] = self.advance()
        return self.u, self.t

    def advance(self):
        """Advance the solution one time step."""
        # Create local variables to get rid of "self." in
        # the numerical formula
        u, f, n, t = self.u, self.f, self.n, self.t
        #dt is not necessarily constant:
        dt = t[n+1]-t[n]
        unew = u[n] + dt*f(u[n], t[n])
        return unew

This class is quite similar to the one above, but we have simplified the
constructor considerably, introduced a separate method for setting the initial
condition, and changed the `solve` method to take an array of time points
as argument. The latter gives a bit more flexibility than the version in
`ForwardEuler_v1`, where the stop time and number of time points were passed
as arguments to the constructor and used to compute a (constant) time step `dt`.
The `ForwardEuler_v2` version does not require the time step to be constant, and
the method will work fine if we pass it a `time_points` array with varying
distance between the time points. This can be useful if we know that the solution
varies rapidly in some time intervals and more slowly in others. However, in most cases
we will use an evenly spaced array for the `time_points` argument, for instance created using
NumPy's `linspace`, and in such cases there is not much difference between the two
FE classes. To consider a concrete example, the solution of the same logistic
growth problem as above using the new class may look as follows:

In [8]:
problem = Logistic(0.2, 1, 0.1)
time = np.linspace(0,40,401)

method = ForwardEuler_v2(problem)
method.set_initial_condition(problem.U0)
u, t = method.solve(time)

# Alternative ODE solvers
As mentioned above, the FE method is not the most sophisticated ODE solver,
although it is sufficiently accurate
for most of the applications we will consider here. Many alternative methods exist, with better accuracy and stability than
FE. One very popular class of ODE solvers is known as Runge-Kutta methods. The simplest example of a Runge-Kutta method
is in fact the FE method;

$$
u_{n+1} = u_n + \Delta t\, f(u_n, t_n),
$$

which is an example of a one-stage, first-order, explicit Runge-Kutta method.
Being a first-order method, the error in the approximate solution produced by
FE is proportional to $\Delta t$. We can easily improve the accuracy to
second order, i.e., error proportional to $\Delta t^2$, by taking one additional step:

$$
\begin{align*}
k_1 & = f(u_n, t_n), \\
k_2 & = f(u_n+\frac{\Delta t}{2} k_1, t_n+\frac{\Delta t}{2}), \\
u_{n+1} &= u_n + \Delta t\, k_2 .
\end{align*}
$$

This method is known as the explicit midpoint method or the modified Euler method. We
see that we first compute an intermediate value, often referred to as a *stage derivative*,
which is an approximation of the derivative at time $t_n$. Then, instead of using this
value to advance the solution to the next step, which we did in the FE method,
we use it to approximate the solution at time $t_n+\Delta t/2$, evaluate the
right hand side function again at this midpoint, and use the result ($k_2$) to
advance the solution to $t_{n+1}$. An alternative second order method is Heun's method,
also referred to as the explicit trapezoidal method:

$$
\begin{align*}
k_1 & = f(u_n, t_n), \\
k_2 & = f(u_n+\Delta t k_1, t_n+\Delta t), \\
u_{n+1} &= u_n + \Delta t\, (k_1/2 + k_2/2)
\end{align*}
$$

This method first approximates the derivative at $t_n$ and $t_{n=1}$, and then
advances the solution using the average of these two.

All Runge-Kutta methods follow the same recipe as the two second order methods considered
above; we compute one or more intermediate values (stage derivatives), and advance the
solution using a combination of these stage derivatives. The accuracy of the method can
be improved by adding more stages. A general RK method with $s$ stages
can be written as

<!-- Equation labels as ordinary links -->
<div id="genrk0"></div>

$$
\begin{equation}
k_i = f(t_n+c_i\Delta t,y_n+\Delta t
\sum_{j=1}^s a_{ij}k_j),  \mbox{\  for } i = 1,\ldots ,s \label{genrk0} \tag{3}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="genrk1"></div>

$$
\begin{equation} 
u_{n+1} = u_0 + \Delta t \sum_{i=1}^s b_i k_i .\label{genrk1} \tag{4}
\end{equation}
$$

Here $c_i, b_i, a_{ij}$, for $i,j, = 1,\ldots ,s$ are
method-specific, given coefficients. All RK methods can be written
in this form, and a method is uniquely determined by the
number of stages $s$ and the values of the coefficients. In the
ODE literature one often sees these coefficients specified in the
form of a *Butcher tableau*, which offers a compact definition of any RK method.
The Butcher tableau is simply a specification of all the method coefficients, and
for a general RK method it is written as

$$
\begin{array}{c|ccc}
c_i & a_{11} & \cdots & a_{1s}\\
\vdots & \vdots & & \vdots \\
c_s & a_{s1} & \cdots & a_{ss} \\ \hline
 & b_1 & \cdots & b_s
\end{array} .
$$

The Butcher tableaus of the three methods considered above; FE, explicit midpoint, and Heun's method, are

$$
\begin{array}{c|cc}
0 & 0 \\ \hline
 & 1
\end{array} \hspace{1cm}
\begin{array}{c|cc}
0 & 0& 0 \\
1/2 & 1/2 & 0 \\ \hline
 & 0 & 1
\end{array} \hspace{1cm}
\begin{array}{c|cc}
0 & 0& 0 \\
1 & 1 & 0 \\ \hline
 & 1/2 & 1/2
\end{array} ,
$$

respectively. To grasp the concept of Butcher tableaus, it is a good exercise to
insert the coefficients from these three tableaus into ([3](#genrk0))-([4](#genrk1))
and verify that the you arrive at the correct formulae for the three methods.
As an example of a method of higher order, we may consider the the
fourth order, four-stage method defined by the Butcher tableau

$$
\begin{array}{c|cccc}
0 & 0& 0 & 0 & 0\\
1/2 & 1/2 & 0 & 0 & 0\\
1/2 & 0 & 1/2 & 0 & 0 \\
1  & 0 & 0 & 1 & 0 \\ \hline
 & 1/6 & 1/3 & 1/3 & 1/6
\end{array}
$$

which gives the formulas

$$
\begin{align*}
k_1 &= f(u_n, t_n) ,\\
k_2 &= f(u_n + {1\over2}k_1, t_n + {1\over2}\Delta t),\\
k_3 &= f(u_n + {1\over2}k_2, t_n + {1\over2}\Delta t),\\
k_4 &= f(u_n + k_3, t_n + \Delta t),
u_{n+1} = u_n + \frac{\Delta t}{6} \left( k_1 + 2k_2 + 2k_3 + k_4\right).
\end{align*}
$$

All the RK methods we will consider in this course are explicit methods, %IN1900
which means that $a_{ij} = 0$ for $j\geq i$. If we examine the formula in ([3](#genrk0)),
we see that the expression for computing each stage derivative $k_i$ then only includes
previously computed stage derivatives, and they can be computed sequentially
from explicit formulas. For implicit RK methods, on the other hand, we
we have $a_{ij} \neq 0$ for some $j\geq i$, and we see in ([3](#genrk0)) that the
formula for computing $k_i$ will then include $k_i$ on the right hand side. We
therefore need to solve equations to compute the stage derivatives, and for non-linear
ODEs these will be non-linear equations that are typically solved using Newton's method.
This makes implicit
RK methods more complex to implement and more computationally expensive per time step,
but they are also more stable than explicit methods and perform much better
for certain classes of ODEs. We will not consider implicit RK methods in this course. %IN1900

# A class hierarchy of ODE solvers
We now want to implement some of the Runge-Kutta methods as classes,
similar to the FE classes introduced above. When inspecting the `ForwardEuler_v2`
class, we quickly observe that most of the code is common to all ODE solvers, and
not specific to the FE method. For instance, we
always need to create an array for holding the solution,
and the general solution method using a for-loop is always the same. In fact, the only
difference between the different methods is how the solution is advanced from one
step to the next. Recalling the ideas of Object-Oriented Programming, it becomes
obvious that a class hierarchy is very convenient for implementing such a collection
of ODE solvers. In this way we can collect all common code in a superclass, and rely on
inheritance to avoid code duplication. The superclass can handle most of the
more "administrative" steps of the ODE solver, such as
  * Storing the solution $u_n$ and the corresponding time levels $t_n$, $k=0,1,2,\ldots,n$

  * Storing the right-hand side function $f(u,t)$

  * Storing and applying initial condition

  * Running the loop over all time steps

We can introduce a superclass `ODESolver` to handle these parts, and implement
the method-specific details in sub-classes. It should now become quite obvious why
we chose to isolate the code to perform a single step in the `advance` method,
since this will then be the only method we need to implement in the subclasses.
The implementation of the superclass may look like

In [9]:
import numpy as np

class ODESolver:
    def __init__(self, f):
        self.f = f

    def advance(self):
        """Advance solution one time step."""
        raise NotImplementedError  # implement in subclass

    def set_initial_condition(self, U0):
        self.U0 = float(U0)

    def solve(self, time_points):
        self.t = np.asarray(time_points)
        N = len(self.t)
        self.u = np.zeros(N)
        # Assume that self.t[0] corresponds to self.U0
        self.u[0] = self.U0

        # Time loop
        for n in range(N-1):
            self.n = n
            self.u[n+1] = self.advance()
        return self.u, self.t

Notice that the `ODESolver` is meant to be a pure superclass, and we have therefore
implemented the `advance`-method to raise an exception if the class is used on its own.
We could also have omitted the `advance`-method from the superclass altogether, but the
chosen implementation makes it clearer to users of the class that `ODESolver`
is a pure superclass.

With the `ODESolver` superclass at hand, the implementation of a `ForwardEuler` class becomes very simple:

In [10]:
class ForwardEuler(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t

        dt = t[n+1] - t[n]
        unew = u[n] + dt*f(u[n], t[n])
        return unew

Similarly, the explicit midpoint method and the fourth-order RK method can be subclasses, each
with a single method:

In [11]:
class ExplicitMidpoint(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = t[n+1] - t[n]
        dt2 = dt/2.0
        k1 = f(u[n], t)
        k2 = f(u[n] + dt2*k1, t[n] + dt2)
        unew = u[n] + dt*k2
        return unew

class RungeKutta4(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = t[n+1] - t[n]
        dt2 = dt/2.0
        k1 = f(u[n], t)
        k2 = f(u[n] + dt2*k1, t[n] + dt2)
        k3 = f(u[n] + dt2*k2, t[n] + dt2)
        k4 = f(u[n] + dt*k3, t[n] + dt)
        unew = u[n] + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
        return unew

The use of these classes is nearly identical to the second version of the FE class introduced
earlier.
Considering
the same simple ODE used above; $u'= u$, $u(0)=1$, $t\in [0,3]$, $\Delta t=0.1$,
the code looks like:

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from ODESolver import ForwardEuler, ExplicitMidpoint, RungeKutta4

def f(u, t):
    return u

time_points = np.linspace(0, 3, 11)

fe = ForwardEuler(f)
fe.set_initial_condition(U0=1)
u1, t1 = fe.solve(time_points)
plt.plot(t1, u1, label='Forward Euler')

em = ExplicitMidpoint(f)
em.set_initial_condition(U0=1)
u2, t2 = em.solve(time_points)
plt.plot(t2, u2, label='Explicit Midpoint')

rk4 = RungeKutta4(f)
rk4.set_initial_condition(U0=1)
u3, t3 = rk4.solve(time_points)
plt.plot(t3, u3, label='RungeKutta 4')

#plot the exact solution in the same plot
time_exact = np.linspace(0,3,301) #more points to improve the plot
plt.plot(time_exact,np.exp(time_exact),label='Exact')

plt.legend()
plt.show()

This code will solve the same equation using three different methods, and plot the solutions
in the same window. Experimenting with different step sizes should reveal the
difference in accuracy between the two methods.